# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key= g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
cities_pd = pd.read_csv("output_data/cities.csv")
cities_pd.head()

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Minna,9.6139,6.5569,79.65,66,100,5.39,NG,1650939128
1,Jalu,29.0331,21.5482,73.49,20,100,9.44,LY,1650939039
2,Rikitea,-23.1203,-134.9692,75.94,74,96,19.35,PF,1650939129
3,Acaraú,-2.8856,-40.1200,77.74,86,49,10.42,BR,1650939129
4,Port Elizabeth,-33.9180,25.5701,57.31,82,0,7.00,ZA,1650939130


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Locations are Lat and Lon from cities_pd
locations = cities_pd[["Lat","Lon"]]
# Humidity is the weight
humidity = cities_pd["Humidity"].astype(float)

In [10]:
# Create the map
fig = gmaps.figure(map_type = "TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=10,point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display the may
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Limiting cities_pd to cities that only have the following values:
# Max Temp values 70-85 degrees
# Humidity values 60-80
# Cloudiness values < 15
locations = cities_pd.loc[(cities_pd['Max Temp'] > 70.0) & (cities_pd['Max Temp'] <= 85.0) & \
                          (cities_pd["Humidity"] >= 60) & (cities_pd["Humidity"] <= 80) & \
                          (cities_pd["Cloudiness"] < 10)]
locations

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
108,São Miguel do Araguaia,-13.2750,-50.1628,75.15,64,6,2.06,BR,1650939155
192,Verāval,20.9000,70.3667,81.28,68,0,13.94,IN,1650939175
235,Jati,24.3539,68.2671,81.09,72,0,12.53,PK,1650939188
251,Haimen,23.1935,116.6142,82.18,71,1,9.71,CN,1650939193
286,Road Town,18.4167,-64.6167,75.29,70,7,13.49,VG,1650939071
303,Marsh Harbour,26.5412,-77.0636,74.23,69,0,16.35,BS,1650939206
326,Saint-Pierre,-21.3393,55.4781,71.28,74,0,8.05,RE,1650939214
365,Charlottesville,38.0293,-78.4767,70.45,75,0,4.61,US,1650939224
393,Tavares,28.8042,-81.7256,75.99,74,0,10.36,US,1650938958
402,Savannah Bight,16.4500,-85.8500,81.03,78,0,19.44,HN,1650939233


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = locations.copy().reset_index(drop=True)
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,São Miguel do Araguaia,-13.2750,-50.1628,75.15,64,6,2.06,BR,1650939155
1,Verāval,20.9000,70.3667,81.28,68,0,13.94,IN,1650939175
2,Jati,24.3539,68.2671,81.09,72,0,12.53,PK,1650939188
3,Haimen,23.1935,116.6142,82.18,71,1,9.71,CN,1650939193
4,Road Town,18.4167,-64.6167,75.29,70,7,13.49,VG,1650939071
5,Marsh Harbour,26.5412,-77.0636,74.23,69,0,16.35,BS,1650939206
6,Saint-Pierre,-21.3393,55.4781,71.28,74,0,8.05,RE,1650939214
7,Charlottesville,38.0293,-78.4767,70.45,75,0,4.61,US,1650939224
8,Tavares,28.8042,-81.7256,75.99,74,0,10.36,US,1650938958
9,Savannah Bight,16.4500,-85.8500,81.03,78,0,19.44,HN,1650939233


In [16]:
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,São Miguel do Araguaia,-13.2750,-50.1628,75.15,64,6,2.06,BR,1650939155,
1,Verāval,20.9000,70.3667,81.28,68,0,13.94,IN,1650939175,
2,Jati,24.3539,68.2671,81.09,72,0,12.53,PK,1650939188,
3,Haimen,23.1935,116.6142,82.18,71,1,9.71,CN,1650939193,
4,Road Town,18.4167,-64.6167,75.29,70,7,13.49,VG,1650939071,
5,Marsh Harbour,26.5412,-77.0636,74.23,69,0,16.35,BS,1650939206,
6,Saint-Pierre,-21.3393,55.4781,71.28,74,0,8.05,RE,1650939214,
7,Charlottesville,38.0293,-78.4767,70.45,75,0,4.61,US,1650939224,
8,Tavares,28.8042,-81.7256,75.99,74,0,10.36,US,1650938958,
9,Savannah Bight,16.4500,-85.8500,81.03,78,0,19.44,HN,1650939233,


In [17]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'types':"hotel",
    'radius':5000,
    'keyword':"hotel",
    'key':g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lon"]
    
    params['location'] = f"{lat},{lon}"
    
    hotel_response = requests.get(base_url,params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"] = hotel_response["results"][0]["name"]
        print(f"Found hotel in {row['City']}")
    except:
        print("No hotel found, moving to next city")

Found hotel in São Miguel do Araguaia
Found hotel in Verāval
Found hotel in Jati
No hotel found, moving to next city
Found hotel in Road Town
Found hotel in Marsh Harbour
Found hotel in Saint-Pierre
Found hotel in Charlottesville
Found hotel in Tavares
Found hotel in Savannah Bight
Found hotel in Dillon
Found hotel in Bubaque
Found hotel in Cumberland
Found hotel in Tura


In [18]:
hotel_df

,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,São Miguel do Araguaia,-13.2750,-50.1628,75.15,64,6,2.06,BR,1650939155,Hotel São Miguel
1,Verāval,20.9000,70.3667,81.28,68,0,13.94,IN,1650939175,Lords Inn Somnath
2,Jati,24.3539,68.2671,81.09,72,0,12.53,PK,1650939188,Temra Hotel Jati
3,Haimen,23.1935,116.6142,82.18,71,1,9.71,CN,1650939193,
4,Road Town,18.4167,-64.6167,75.29,70,7,13.49,VG,1650939071,Maria's By The Sea Hotel
5,Marsh Harbour,26.5412,-77.0636,74.23,69,0,16.35,BS,1650939206,Abaco Beach Resort
6,Saint-Pierre,-21.3393,55.4781,71.28,74,0,8.05,RE,1650939214,Hôtel Le Saint-Pierre
7,Charlottesville,38.0293,-78.4767,70.45,75,0,4.61,US,1650939224,Hyatt Place Charlottesville
8,Tavares,28.8042,-81.7256,75.99,74,0,10.36,US,1650938958,Key West Resort on Lake Dora - Tavares
9,Savannah Bight,16.4500,-85.8500,81.03,78,0,19.44,HN,1650939233,Guanaja Beach Hotel


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [20]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Add marker layer ontop of heat map
hotel_data_layer = gmaps.marker_layer(locations,info_box_content = \
                                      [info_box_template.format(**row) for index, row in hotel_df.iterrows()])
fig.add_layer(hotel_data_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))